In [856]:
import pandas as pd
import numpy as np
import itertools as it
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

In [857]:
data = pd.read_csv("train.csv")
submission_input = pd.read_csv("test.csv")

In [858]:
print(data.head())
print(len(data.Parch))

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  
89

In [859]:
def convert_cabin(x):
    if x != x:
        return 0
    return ord(x[0]) - ord('B')

def convert_embarked(x):
    if x == 'Q':
        return 1
    if x == 'S':
        return 2
    return 0


In [860]:
def clean(data_frame: pd.DataFrame):
    data_frame.drop(['Name','Ticket', 'PassengerId','Cabin', 'Embarked', 'Fare'], axis=1, inplace=True)
    data_frame.dropna()
    data_frame.Age = data_frame.Age.fillna(data_frame.Age.median())
    data_frame.Sex = pd.get_dummies(data_frame.Sex, drop_first=True)
   


In [861]:
pId = submission_input.PassengerId
clean(submission_input)
clean(data)

In [862]:
print(data.head())
print(len(data.Parch))

   Survived  Pclass  Sex   Age  SibSp  Parch
0         0       3    1  22.0      1      0
1         1       1    0  38.0      1      0
2         1       3    0  26.0      0      0
3         1       1    0  35.0      1      0
4         0       3    1  35.0      0      0
891


In [863]:
def shuffled_data(data):
    data = data.sample(frac=1)
    train, test = train_test_split(data, test_size=0.2)
    return (train.drop(['Survived'],axis=1),train['Survived'],test.drop(['Survived'], axis=1),test['Survived'])

In [864]:
accuracies = {}

In [865]:
def train_model(m,c,k, x, y, test_x, test_y):
    model = DecisionTreeClassifier(max_depth=m,criterion=c,min_weight_fraction_leaf=k)
    model.fit(x, y)
    pre = model.predict(test_x)
    return accuracy_score(pre,test_y)

In [866]:
def cl(x):
    values = [0.74,0.76,0.78, 0.79,0.80,0.81, 0.82,0.83, 0.84, 0.85,0.86, 2]
    result = {}

    for i in values:
        result[i] = 0
    for k in x: 
        for i in values:
            if(k < i):
                result[i] += 1
                break
    return result


In [867]:
def sco(x):
    r = 0
    j = 1
    for i in x.items():
        r += j*i[1]*i[0]
        j+=10
    return r

In [868]:
ac= {}
for i in range(0, 100):
    d = shuffled_data(data)
    for a in it.product(range(10,101, 10),['gini', 'entropy', 'log_loss'],[0.01]):
        ac.setdefault(a, [])
        ac[a].append(train_model(*a, *d))


In [869]:
for i in ac.items():
    print(i[0],sco(cl(i[1])))

(10, 'gini', 0.01) 5522.58
(10, 'entropy', 0.01) 6093.709999999999
(10, 'log_loss', 0.01) 6120.44
(20, 'gini', 0.01) 5487.34
(20, 'entropy', 0.01) 5985.3
(20, 'log_loss', 0.01) 5993.81
(30, 'gini', 0.01) 5513.47
(30, 'entropy', 0.01) 5976.59
(30, 'log_loss', 0.01) 6002.92
(40, 'gini', 0.01) 5495.450000000001
(40, 'entropy', 0.01) 5967.48
(40, 'log_loss', 0.01) 5976.19
(50, 'gini', 0.01) 5522.58
(50, 'entropy', 0.01) 5975.79
(50, 'log_loss', 0.01) 5976.19
(60, 'gini', 0.01) 5522.58
(60, 'entropy', 0.01) 5967.28
(60, 'log_loss', 0.01) 5976.59
(70, 'gini', 0.01) 5469.320000000001
(70, 'entropy', 0.01) 5976.19
(70, 'log_loss', 0.01) 5967.48
(80, 'gini', 0.01) 5495.450000000001
(80, 'entropy', 0.01) 5975.79
(80, 'log_loss', 0.01) 5985.1
(90, 'gini', 0.01) 5469.320000000001
(90, 'entropy', 0.01) 5984.9
(90, 'log_loss', 0.01) 5985.1
(100, 'gini', 0.01) 5522.58
(100, 'entropy', 0.01) 5967.08
(100, 'log_loss', 0.01) 5985.3


In [870]:
for k in accuracies.items():
    print(k)

In [871]:
np.argmax([sco(cl(x)) for x in ac.values()])


2

In [872]:
np.argmax([np.mean(x) for x in ac.values()])

8

In [873]:
[x for x in accuracies.items()][np.argmax([x[0] for x in accuracies.values()])]


ValueError: attempt to get argmax of an empty sequence

In [ ]:
# acc = 0
model = DecisionTreeClassifier(max_depth=6,criterion='entropy',min_weight_fraction_leaf=0.01)
# while(acc < 0.9):
#     x, y, x_test, y_test = shuffled_data(data)
#     model.fit(x,y)
#     pre = model.predict(x_test)
#     acc = accuracy_score(pre,y_test)
# acc

In [ ]:
x, y, x_test, y_test = shuffled_data(data)
model.fit(x,y)
pre = model.predict(x_test)
accuracy_score(pre,y_test)

0.770949720670391

In [ ]:
submission = pd.DataFrame({"PassengerId":pId, "Survived":model.predict(submission_input)})

In [ ]:
submission.to_csv("submission.csv", mode='w', index=False)

In [ ]:
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
